In [1]:
import pandas as pd, numpy as np
import sqlite3, nltk, string, math, pickle, sys

from sklearn.model_selection import KFold
from sklearn import svm, linear_model, ensemble
from sklearn.model_selection import cross_val_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from mlxtend.classifier import StackingClassifier
from csel import csel_model

from pycm import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_sample_responses(vectors, responses):
    samples = vectors
    index = list(set(samples.index).intersection(set(labels.index)))
    responses = responses.loc[index]
    samples = samples.loc[index]
    return samples, responses

In [3]:
labels = pd.read_csv("lable(3.1).csv", index_col=0 )
data = pd.read_csv("mibig(3.1).csv", index_col=0 )

In [4]:
%%time

X , y = get_sample_responses(data, labels)
bgc_ids = pd.Series(y.index)
model = csel_model()

metrics = []
for r in range(10):
    all_summary = []
    class_summary = []
    cv = KFold(n_splits=5,random_state=r,shuffle=True)
    for i, (id_train, id_val) in enumerate(cv.split(bgc_ids)):
        train_ids, val_ids = bgc_ids[id_train], bgc_ids[id_val]
        X_train, X_val = X.loc[train_ids].values, X.loc[val_ids].values
        y_train, y_val = y.loc[train_ids].values, y.loc[val_ids].values
        print('Train:', len(X_train), 'Test:', len(X_val))
    
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        y_true = y_val.flatten()
    
        cm = ConfusionMatrix(actual_vector=y_true, predict_vector=y_pred)
   
        all_columns = pd.DataFrame(cm.overall_stat).columns
        all_ = pd.DataFrame(cm.overall_stat)[1:2].values
        all_summary.append(all_[0])
    
        class_columns = pd.DataFrame(cm.class_stat).columns
        class_ = pd.DataFrame(cm.class_stat)[1:2].values
        class_summary.append(class_[0])   

    all_stat = pd.DataFrame(all_summary,columns=[all_columns]).mean()
    all_stat = all_stat.T
    class_stat = pd.DataFrame(class_summary,columns=[class_columns]).mean()
    class_stat = class_stat.T

    metric = pd.concat([all_stat, class_stat], axis=0)
    
    metrics.append(metric.values)
    
metrics = pd.DataFrame(metrics,columns=[metric.index.get_level_values(level=0)])    
    
metrics.to_csv("metrics_CSEL_HPO.csv")

Train: 898 Test: 225
Model starts...
--------------------------------
layer : 1 | acc : 0.766667
layer : 2 | acc : 0.761111
Train: 898 Test: 225
Model starts...
--------------------------------
layer : 1 | acc : 0.711111
layer : 2 | acc : 0.722222
layer : 3 | acc : 0.716667
Train: 898 Test: 225
Model starts...
--------------------------------
layer : 1 | acc : 0.700000
layer : 2 | acc : 0.705556
layer : 3 | acc : 0.705556
Train: 899 Test: 224
Model starts...
--------------------------------
layer : 1 | acc : 0.750000
layer : 2 | acc : 0.755556
layer : 3 | acc : 0.750000
Train: 899 Test: 224
Model starts...
--------------------------------
layer : 1 | acc : 0.716667
layer : 2 | acc : 0.705556
Train: 898 Test: 225
Model starts...
--------------------------------
layer : 1 | acc : 0.750000
layer : 2 | acc : 0.750000
Train: 898 Test: 225
Model starts...
--------------------------------
layer : 1 | acc : 0.677778
layer : 2 | acc : 0.661111
Train: 898 Test: 225
Model starts...
--------------